# Exploring Agave Using the agavepy Libarary

In this notebook, we introduce some basic uses of the agavepy Python library for interacting with the Agave Platform science-as-a-service APIs. The examples primarily draw from the apps service, but the concepts introduced are broadly applicable to all Agave services. In subsequent notebooks, we'll take deeper dives into specific topics such as using agavepy to launch and monitor an Agave job. For more information about Agave, please see the developer site: http://agaveapi.co/

The agavepy library provides a high-level Python binding to the Agave API. The first step is to import the Agave class:

In [34]:
from agavepy.agave import Agave

Before we can interact with Agave, we need to instantiate a client. Typically, we would use the constructor and pass in our credentials (OAuth client key and secret as well as our username and password) together with configuration data for our "tenant", the organization within Agave we wish to interact with. 

The agavepy library's Agave class also provides a restore() method for reconstituting previous OAuth sessions. Previous sessions are read from and written to a cache file, /etc/.agpy, so that OAuth sessions persist across iPython sessions. When you authenticated to JupyterHub, the OAuth login was written to the .agpy file. We can therefore use the restore method to create an OAuth client without needing to pass any credentials:

In [35]:
ag = Agave.restore()

Note that the restore method can take arguments (such as client_name) so that you can restore/manage multiple OAuth sessions. When first getting started on the hub, there is only one session in the cache file, so no arguments are required. 

If we ever want to inspect the OAuth session being used by our client, we have a few methods available to us. First, we can print the token_info dictionary on the token object:

In [36]:
ag.token.token_info

{'access_token': u'39b5c847e62f2814ac8938f42e961a',
 'refresh_token': u'564f25643448c1e5b225bfed32a65ec'}

This shows us both the access and refresh tokens being used. We can also see the end user profile associated with these tokens:

In [37]:
ag.profiles.get()

{u'create_time': u'20140515180254Z',
 u'email': u'jstubbs@tacc.utexas.edu',
 u'first_name': u'Joe',
 u'full_name': u'jstubbs',
 u'last_name': u'Stubbs',
 u'mobile_phone': u'',
 u'phone': u'',
 u'status': u'',
 u'username': u'jstubbs'}

Finally, we can inspect the ag object directly for attributes like api_key, api_secret, api_server, etc.

In [38]:
print ag.api_key, ag.api_secret, ag.api_server

3J_tyBc6329PxHtyYMSewFJPyNsa cboMOXUFfDJAO7CV68u0KFUStbca https://agave.designsafe-ci.org


We are now ready to interact with Agave services using our agavepy client. We can take a quick look at the available top-level methods of our client:

In [39]:
dir(ag)

[u'apps',
 u'clients',
 u'files',
 u'jobs',
 u'meta',
 u'monitors',
 u'notifications',
 u'postits',
 u'profiles',
 u'systems',
 u'transforms']

We see there is a top-level method for each of the core science APIs in agave. We will focus on the apps service since it is of broad interest, but much of what we illustrate is generally applicable to all Agave core science APIs. 

We can browse a specific collection using the list() method. For example, let's see what apps are available to us:

In [40]:
ag.apps.list()

[{u'_links': {u'self': {u'href': u'https://agave.designsafe-ci.org/apps/v2/adcirc-parallel-1920-51.33'}},
  u'executionSystem': u'designsafe.community.exec.stampede.nores',
  u'id': u'adcirc-parallel-1920-51.33',
  u'isPublic': False,
  u'label': u'Parallel ADCIRC 1920',
  u'lastModified': datetime.datetime(2016, 8, 22, 15, 44, 20, tzinfo=tzoffset(None, -18000)),
  u'name': u'adcirc-parallel-1920',
  u'revision': 1,
  u'shortDescription': u'Parallel ADCIRC is for solving systems of shallow water equations with applications to hurricane storm surge modeling.',
  u'version': u'51.33'},
 {u'_links': {u'self': {u'href': u'https://agave.designsafe-ci.org/apps/v2/Matlab-batch-0.3'}},
  u'executionSystem': u'designsafe.community.exec.stampede',
  u'id': u'Matlab-batch-0.3',
  u'isPublic': False,
  u'label': u'MATLAB (batch)',
  u'lastModified': datetime.datetime(2016, 8, 18, 13, 44, 24, tzinfo=tzoffset(None, -18000)),
  u'name': u'Matlab-batch',
  u'revision': 7,
  u'shortDescription': u'Run 

What we see in the output above is a python list representing the JSON object returned from Agave's apps service. It is a list of objects, each of which representing a single app. Let's capture the first app object and inspect it. To do that we can use normal Python list notation:

In [41]:
app = ag.apps.list()[0]

In [42]:
print type(app); app

<class 'agavepy.agave.AttrDict'>


{u'_links': {u'self': {u'href': u'https://agave.designsafe-ci.org/apps/v2/adcirc-parallel-1920-51.33'}},
 u'executionSystem': u'designsafe.community.exec.stampede.nores',
 u'id': u'adcirc-parallel-1920-51.33',
 u'isPublic': False,
 u'label': u'Parallel ADCIRC 1920',
 u'lastModified': datetime.datetime(2016, 8, 22, 15, 44, 20, tzinfo=tzoffset(None, -18000)),
 u'name': u'adcirc-parallel-1920',
 u'revision': 1,
 u'shortDescription': u'Parallel ADCIRC is for solving systems of shallow water equations with applications to hurricane storm surge modeling.',
 u'version': u'51.33'}

We see that the app object is of type agavepy.agave.AttrDict. That's a Python dictionary with some customizations to provide convenience features such as using dot notation for keys/attributes. For example, we see that the app object has an 'id' key. We can access it directly using dot notation:

In [43]:
app.id

u'adcirc-parallel-1920-51.33'

Equivalently, we can use normal Python dictionary syntax:

In [44]:
app['id']

u'adcirc-parallel-1920-51.33'

In Agave, the app id is the unique identifier for the application. We'll come back to that in a minute. For now, just know that this example is very typical of responses from agavepy: in general the JSON response object is represented by lists of AttrDicts.

Stepping back for a second, let's explore the apps collection a bit. We can always get a list of operations available for a collection by using the dir(-) method:

In [45]:
dir(ag.apps)

[u'add',
 u'delete',
 u'deletePermissions',
 u'deletePermissionsForUser',
 u'get',
 u'getJobSubmissionForm',
 u'list',
 u'listByName',
 u'listByOntologyTerm',
 u'listBySystemId',
 u'listByTag',
 u'listPermissions',
 u'listPermissionsForUser',
 u'manage',
 u'update',
 u'updateApplicationPermissions',
 u'updatePermissionsForUser']

Also notice that we have tab-completion on these operations. So, if we start typing "ag.apps.l" and then hit tab, Jupyter provides a select box of operations beginning with "l". Try putting the following cell in focus and then hitting the tab key (but don't actually hit enter or try to execute the cell; otherwise you'll get an exception because there's no method called "l"):

In [ ]:
ag.apps.l

If we would like to get details about a specific object for which we know the unique id, in general we use the get method, passing in the id for the object. Here, we will use an app id we found from the ag.apps.list command.

In [47]:
ag.apps.get(appId=app.id)

{u'_links': {u'executionSystem': {u'href': u'https://agave.designsafe-ci.org/systems/v2/designsafe.community.exec.stampede.nores'},
  u'history': {u'href': u'https://agave.designsafe-ci.org/apps/v2/adcirc-parallel-1920-51.33/history'},
  u'metadata': {u'href': u'https://agave.designsafe-ci.org/meta/v2/data/?q=%7B%22associationIds%22%3A%223709002573100421606-242ac116-0001-005%22%7D'},
  u'owner': {u'href': u'https://agave.designsafe-ci.org/profiles/v2/mock'},
  u'permissions': {u'href': u'https://agave.designsafe-ci.org/apps/v2/adcirc-parallel-1920-51.33/pems'},
  u'self': {u'href': u'https://agave.designsafe-ci.org/apps/v2/adcirc-parallel-1920-51.33'},
  u'storageSystem': {u'href': u'https://agave.designsafe-ci.org/systems/v2/designsafe.storage.default'}},
 u'available': True,
 u'checkpointable': False,
 u'defaultMaxRunTime': u'04:00:00',
 u'defaultMemoryPerNode': 1,
 u'defaultNodeCount': 120,
 u'defaultProcessorsPerNode': 1920,
 u'defaultQueue': u'normal',
 u'deploymentPath': u'mock/a

Whoa, that's a lot of information. We aren't going to give a comprehensive introduction to Agave applications in this notebook. Instead we refer you to the official Agave app tutorial on the website: http://agaveapi.co/documentation/tutorials/app-management-tutorial/

However, we will point out a couple of important points. Let's capture that response in an object called full_app:

In [48]:
full_app = ag.apps.get(appId=app.id)

Complex sub-objects of the application such as application inputs and parameters come back as top level attributes. and are represented as lists. The individual elements of the list are represented as AttrDicts. We can see this by exploring our full_app's inputs:

In [49]:
print type(full_app.inputs); print type(full_app.inputs[0]); full_app.inputs[0]

<type 'list'>
<class 'agavepy.agave.AttrDict'>


{u'details': {u'argument': None,
  u'description': u"The directory containing your ADCIRC input files. You can drag the link for the directory from the Data Browser on the left, or click the 'Select Input' button and then select the directory. To try this out with example data, copy and paste 'agave://designsafe.storage.default/mock/examples/adcirc/EC2001' above.",
  u'label': u'Input Directory',
  u'repeatArgument': False,
  u'showArgument': False},
 u'id': u'inputDirectory',
 u'semantics': {u'fileTypes': [u'raw-0'],
  u'maxCardinality': 1,
  u'minCardinality': 1,
  u'ontology': [u'xsd:string']},
 u'value': {u'default': u'agave://designsafe.storage.default/mock/examples/adcirc/EC2001',
  u'enquote': False,
  u'order': 0,
  u'required': True,
  u'validator': u'',
  u'visible': True}}

Then, if we want the input id, we can use dot notation or dictionary notation just as before:

In [50]:
full_app.inputs[0].id

u'inputDirectory'

You now have the ability to fully explore individual Agave objects returned from agavepy, but what about searching for objects? The Agave platform provides a powerful search feature across most services, and agavepy supports that as well. 

Every retrieval operation in agavepy (for example, apps.list) supports a "search" argument. The syntax for the search argument is identical to that described in the Agave documentation: it uses a dot notation combining search terms, values and (optional) operators. The search object itself should be a python dictionary with strings for the keys and values. Formally, each key:value pair in the dictionary adheres to the following form: 
                        $$term.operator:value$$
The operator is optional and defaults to equality ('eq'). For example, the following search filters the list of all apps down to just those with the id attribute equal to our app.id:

In [51]:
ag.apps.list(search={'id': app.id})

[{u'_links': {u'self': {u'href': u'https://agave.designsafe-ci.org/apps/v2/adcirc-parallel-1920-51.33'}},
  u'executionSystem': u'designsafe.community.exec.stampede.nores',
  u'id': u'adcirc-parallel-1920-51.33',
  u'isPublic': False,
  u'label': u'Parallel ADCIRC 1920',
  u'lastModified': datetime.datetime(2016, 8, 22, 15, 44, 20, tzinfo=tzoffset(None, -18000)),
  u'name': u'adcirc-parallel-1920',
  u'revision': 1,
  u'shortDescription': u'Parallel ADCIRC is for solving systems of shallow water equations with applications to hurricane storm surge modeling.',
  u'version': u'51.33'}]

Equivalently, we could explicitly set the equality operator:

In [52]:
ag.apps.list(search={'id.eq': app.id})

[{u'_links': {u'self': {u'href': u'https://agave.designsafe-ci.org/apps/v2/adcirc-parallel-1920-51.33'}},
  u'executionSystem': u'designsafe.community.exec.stampede.nores',
  u'id': u'adcirc-parallel-1920-51.33',
  u'isPublic': False,
  u'label': u'Parallel ADCIRC 1920',
  u'lastModified': datetime.datetime(2016, 8, 22, 15, 44, 20, tzinfo=tzoffset(None, -18000)),
  u'name': u'adcirc-parallel-1920',
  u'revision': 1,
  u'shortDescription': u'Parallel ADCIRC is for solving systems of shallow water equations with applications to hurricane storm surge modeling.',
  u'version': u'51.33'}]

Typically, the list of available search terms is identical to the attributes included in the JSON returned when requesting the full resource description. Operators include 'like', 'lt', 'gt', 'lte', 'gte', etc. See the official Agave documentation for the complete list. 

Here we retrieve all apps with a name is "like" opensees:

In [53]:
ag.apps.list(search={'name.like': 'opensees'})

[{u'_links': {u'self': {u'href': u'https://agave.designsafe-ci.org/apps/v2/opensees-2.5.0.6248'}},
  u'executionSystem': u'designsafe.community.exec.stampede',
  u'id': u'opensees-2.5.0.6248',
  u'isPublic': False,
  u'label': u'OpenSeesSP',
  u'lastModified': datetime.datetime(2016, 3, 25, 12, 6, 53, tzinfo=tzoffset(None, -18000)),
  u'name': u'opensees',
  u'revision': 1,
  u'shortDescription': u'OpenSees is a software framework for simulating the seismic response of structural and geotechnical systems.',
  u'version': u'2.5.0.6248'},
 {u'_links': {u'self': {u'href': u'https://agave.designsafe-ci.org/apps/v2/opensees-2.4.4.5804'}},
  u'executionSystem': u'designsafe.community.exec.stampede',
  u'id': u'opensees-2.4.4.5804',
  u'isPublic': False,
  u'label': u'OpenSeesSP',
  u'lastModified': datetime.datetime(2016, 3, 11, 7, 54, 37, tzinfo=tzoffset(None, -21600)),
  u'name': u'opensees',
  u'revision': 29,
  u'shortDescription': u'OpenSees is a software framework for simulating the se

Two results were returned, both with name "opensees".

You can include multiple search expressions in the form of additional key:value pairs to build a more restrictive query. Here we restrict the result to opensees apps with revision at least 25:

In [54]:
ag.apps.list(search={'name.like': 'opensees', 'revision.gte': 25})

[{u'_links': {u'self': {u'href': u'https://agave.designsafe-ci.org/apps/v2/opensees-2.4.4.5804'}},
  u'executionSystem': u'designsafe.community.exec.stampede',
  u'id': u'opensees-2.4.4.5804',
  u'isPublic': False,
  u'label': u'OpenSeesSP',
  u'lastModified': datetime.datetime(2016, 3, 11, 7, 54, 37, tzinfo=tzoffset(None, -21600)),
  u'name': u'opensees',
  u'revision': 29,
  u'shortDescription': u'OpenSees is a software framework for simulating the seismic response of structural and geotechnical systems.',
  u'version': u'2.4.4.5804'}]

We hope this gives you enough general information to begin exploring the Agave services using agavepy on your own. In subsequent notebooks, we'll take deeper dives into specific topics such as using agavepy to launch and monitor an Agave job executing OpenSees on Stampede.